- Author: Benjamin Du
- Date: 2023-07-01 12:44:50
- Modified: 2023-07-01 12:44:50
- Title: Amazon AWS Ec2 Instances
- Slug: amazon-aws-ec2-instances
- Category: Computer Science
- Tags: Computer Science, programming

**Things on this page are fragmentary and immature notes/thoughts of the author. Please read with your own judgement!**

https://aws.amazon.com/ec2/pricing/on-demand/

![](https://user-images.githubusercontent.com/824507/250304785-7f0487f0-82f2-469e-b1aa-ccb08dcf412a.png)

In [26]:
import polars as pl

In [37]:
df = pl.read_csv("aws_ec2_vm_instances.csv")
df = df.with_columns(
    hourly_cost=pl.col("hourly_cost").str.lstrip("$").cast(float),
    memory=pl.col("memory").str.rstrip(" GiB").cast(float),
    performance_per_vcpu=pl.when(pl.col("performance_per_vcpu").is_null()).then(1).otherwise(
        pl.col("performance_per_vcpu")
    ),
).with_columns(
    real_vcpu=pl.col("vcpu") * pl.col("performance_per_vcpu"),
)
df

name,hourly_cost,vcpu,memory,storage,network,performance_per_vcpu,network_burst,ebs_burst,real_vcpu
str,f64,i64,f64,str,str,f64,str,str,f64
"""a1.medium""",0.0255,1,2.0,"""EBS Only""","""Up to 10 Gigab…",1.0,null,null,1.0
"""a1.large""",0.051,2,4.0,"""EBS Only""","""Up to 10 Gigab…",1.0,null,null,2.0
"""a1.xlarge""",0.102,4,8.0,"""EBS Only""","""Up to 10 Gigab…",1.0,null,null,4.0
"""a1.2xlarge""",0.204,8,16.0,"""EBS Only""","""Up to 10 Gigab…",1.0,null,null,8.0
"""a1.4xlarge""",0.408,16,32.0,"""EBS Only""","""Up to 10 Gigab…",1.0,null,null,16.0
"""a1.metal""",0.408,16,32.0,"""EBS Only""","""Up to 10 Gigab…",1.0,null,null,16.0
"""t4g.nano""",0.0042,2,0.5,"""EBS Only""","""Up to 5 Gigabi…",0.05,"""Up to 5""","""Up to 2,085""",0.1
"""t4g.micro""",0.0084,2,1.0,"""EBS Only""","""Up to 5 Gigabi…",0.1,"""Up to 5""","""Up to 2,085""",0.2
"""t4g.small""",0.0168,2,2.0,"""EBS Only""","""Up to 5 Gigabi…",0.2,"""Up to 5""","""Up to 2,085""",0.4


## VM Instances With Highest vCPU / memory Ratio

In [38]:
(df["real_vcpu"] / df["memory"]).max()

2.0

In [39]:
df.filter(df["real_vcpu"] / df["memory"] >= 2)

name,hourly_cost,vcpu,memory,storage,network,performance_per_vcpu,network_burst,ebs_burst,real_vcpu
str,f64,i64,f64,str,str,f64,str,str,f64
"""t2.nano""",0.0058,1,0.5,"""EBS Only""","""Low""",1.0,null,null,1.0


## Cheapest VM Instance per vCPU Hour

`t2.nano` is the cheapest with an hourly cost of `$0.0058` and a monthly cost of `$4.176` per vCPU.

In [40]:
(df["hourly_cost"] / df["real_vcpu"]).min()

0.0058

In [42]:
df.filter(df["hourly_cost"] / df["real_vcpu"] <= 0.0058)

name,hourly_cost,vcpu,memory,storage,network,performance_per_vcpu,network_burst,ebs_burst,real_vcpu
str,f64,i64,f64,str,str,f64,str,str,f64
"""t2.nano""",0.0058,1,0.5,"""EBS Only""","""Low""",1.0,null,null,1.0


Monthly cost (USD).

In [43]:
0.0058 * 24 * 30

4.176

## GCP

GCP c2 and c3 VMs are good choices according to performance per price.
It's even cheaper than customized VM types with reduced memories!
GCP c2 VMs are compute optimized and are ideal for CPU intensive workloads.
Note that GCP c2 spot VMs are even cheaper than AWS t2.nano VMs
if you can stand your VM being preempted.

![](https://user-images.githubusercontent.com/824507/250313656-891cd99a-c9a0-4004-9158-fd87503c39a6.png)

![](https://user-images.githubusercontent.com/824507/250313771-b776b124-5503-4aa9-a457-a55452bfdf67.png)

## References

AWS vs GCP vs on-premises CPU performance comparison
https://medium.com/infrastructure-adventures/aws-vs-gcp-vs-on-premises-cpu-performance-comparison-1cb3e91f9716

GCP - Create a VM with a custom machine type
https://cloud.google.com/compute/docs/instances/creating-instance-with-custom-machine-type

GCP - Free cloud features and trial offer
https://cloud.google.com/free/docs/free-cloud-features#compute


https://aws.amazon.com/ec2/pricing/on-demand/

GCP - Choosing the right virtual machine type
https://cloud.google.com/compute#section-6

GCP Product Calculator
https://cloud.google.com/products/calculator

GCP - Compute Engine Pricing
https://cloud.google.com/compute/all-pricing

Top 10 Cloud Provider Comparison 2023: VM Performance / Price
https://dev.to/dkechag/cloud-vm-performance-value-comparison-2023-perl-more-1kpp